<h1>Install Dependcies</h1>

In [1]:
!pip install torch
!pip install numpy
!pip install gymnasium
!pip install stable_baselines3[extra]


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached stable_baselines3-2.1.0-py3-none-any.whl (178 kB)



[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<h1>Imports</h1>

In [2]:
import gymnasium as gym
from gym import Env
from gym.spaces import Discrete,MultiDiscrete, Box
import numpy as np

ModuleNotFoundError: No module named 'gym'

<h1>Create Game Classes</h1>

In [3]:
# %load classes.py
import functools
import random
class Domino():
    def __init__(self, s1:int,s2:int):
        self.sides = (s1,s2)
        self.isDouble = (s1 == s2)
    def evalute_side(self, side: int):
        if(self.sides[0] == side): return 0
        elif (self.sides[1] == side): return 1
    def calc_points(self):
        return self.sides[0] + self.sides[1]
    def __str__(self):
        return str(self.sides)


class BoneYard():
    def __init__(self):
        self.dominos = []
        self.build()
        self.shuffle()
    def build(self):
        for i in range(0,13):
            for j in range(i,13):
                self.dominos.append(Domino(i,j))
    def shuffle(self):
        random.shuffle(self.dominos)
    def draw(self):
        if len(self.dominos) == 0: return False
        return self.dominos.pop()


class Train():
    startingSide = 12
    def __init__(self, id):
        self.openSides = [Train.startingSide]
        self.trainUp = False
        self.id = id
    def add(self,placement,domino):
        placeIndex = self.openSides.index(placement)
        trainSide = self.openSides[placeIndex]
        if(trainSide in domino.sides):
            if(domino.isDouble):
                self.openSides.pop(placeIndex)
                self.openSides.append(domino.sides[0])
                self.openSides.append(domino.sides[1])
            else:
                self.openSides.pop(placeIndex)
                self.openSides.append(domino.sides[1-domino.evalute_side(trainSide)])
        else:
            return False
    def __str__(self):
        return f"id: {self.id} trainUp?:{self.trainUp} openSides: {self.openSides}"
        

class Player():
    handSize = 12
    nextID = 0 
    def __init__(self,boneYard:BoneYard):
        self.id = Player.nextID
        Player.nextID +=1
        self.hand = []
        for _ in range(Player.handSize): self.hand.append(boneYard.draw()) 
    def highestDouble(self):
        highest = -1
        for domino in self.hand:
            if(domino.isDouble and domino.sides[0] > highest): highest = domino.sides[0]
        return highest
    def intializeTrain(self):
        if not hasattr(self,"train"):
            self.train = Train(self.id)
        return self.train
    def getDominoFromSides(self,s1:int,s2:int):
        for domino in self.hand:
            if (domino.sides == (s1,s2) or domino.sides == (s2,s1)):
                return domino
    def play(self, domino:Domino, placement:int, train:Train|None=None,firstDouble:bool=False):
        selfTrain = False
        played = False
        if train is None: 
            train = self.train
            selfTrain = True
        if firstDouble: 
            self.hand.remove(domino)
            played = True
        elif train.add(placement, domino) != False:
            self.hand.remove(domino)
            if selfTrain: self.train.trainUp = False
            played = True
        if len(self.hand) <= 0: return None
        else: return played
        

    def pointsInHand(self):
        return functools.reduce(lambda acc, domino: acc + domino.calc_points(), self.hand, 0)
    def pickup(self,boneYard:BoneYard):
        domino = boneYard.draw()
        if domino is not False:
            self.hand.append(domino)
        return domino
    def __str__(self):
        return f"id:{self.id} train:{self.train.id}"
    



class Game():
    def __init__(self,numPlayers:int):
        self.boneyard = BoneYard()
        self.players = []
        self.trains = []
        self.done = False
        self.numPlayers = numPlayers
        self.unsastifiedDouble = None
        if(numPlayers<= 4): Player.handSize = 15
        elif(numPlayers<=6): Player.handSize = 12
        elif(numPlayers<=8): Player.handSize = 10
        Player.nextID = 0
        for _ in range(numPlayers): 
            self.players.append(Player(self.boneyard))
        doubles = [player.highestDouble() for player in self.players]
        highestDouble = max(doubles)
        firstPlayer = doubles.index(highestDouble)
        firstDomino = self.players[firstPlayer].getDominoFromSides(highestDouble,highestDouble)
        for player in self.players: 
            player.intializeTrain()
            self.trains.append(player.train)
        if (len(self.players)<8): self.mexican = Train(8)
        self.centerDouble = max(doubles)
        Train.startingSide = self.centerDouble
        self.players[firstPlayer].play(firstDomino,0,firstDouble=True) #removing first double
        self.startingPlayer = self.nextPlayer(firstPlayer)
        self.prevPlayer = None
    def getTrain(self,id:int):
        if id == 8: return self.mexican
        else:
            for t in self.trains:
                if t.id==id:
                    return t
            else:
                return None
    def getPlayer(self,id:int):
        if id > 7: return None
        else: 
            for p in self.players:
                if p.id==id:
                    return p
            else:
                return None
    def nextPlayer(self, currPlayer:int):
        next = currPlayer+1 
        if (next>=self.numPlayers): next = 0 # looping around if its not an actual player
        return next

class BoardState():
    def __init__(self, trains:list[Train],centerDouble:int, mexican:Train|None = None,unsastifiedDouble=None):
        self.mexican = mexican
        self.trains = trains
        self.unsastifiedDouble = unsastifiedDouble
    #train up returns only sides that are on trains with thier trains up
    #maybe this signature should be changed to just take a list of trains? and let caller deal with filtering?
    def getPlacements(self, trainUp: bool=False,include:list[Train]=[], exclude:list[Train]=[]):
        trains = [*self.trains,self.mexican]
        placements = []
        if trainUp:
            for train in trains:
                if (train in include or train.trainUp) and train not in exclude:
                    for side in train.openSides: 
                        placements.append((train.id,side))
        else:
            for train in trains: 
                if train not in exclude:
                    for side in train.openSides: placements.append((train.id,side))
        return placements
    def getTrain(self, id):
        for train in self.trains:
            if train.id == id: return train
    def availablePlays(self, player:Player,placements:list|None=None):
        plays = []
        places = []
        if self.unsastifiedDouble is not None:
            places = [self.unsastifiedDouble]
        elif placements is not None:
            places = placements
        elif player.train.trainUp:
            places = [(player.id, side) for side in player.train.openSides]
        else: places = self.getPlacements(trainUp=True, include=[player.train])
        for placement in places:
            for domino in player.hand:
                eval = domino.evalute_side(placement[1])
                if( eval is not None): plays.append((domino.sides, placement))
        return plays
    def isValidPlay(self, player:Player, action:list[list]):
        valid = False
        plays = self.availablePlays(player)
        for play in plays:
            tuplist = [tuple(list) for list in action]
            if tuple(tuplist) == play:
                valid = True
                print(f"valid play: {play}")
        return valid
    @staticmethod
    def fromGame(game:Game):
        return BoardState(game.trains,game.centerDouble, game.mexican,game.unsastifiedDouble)


<h2>Test Game Objects</h2>

<h3>Testing intialization</h3>

In [31]:
game = Game(6)
bs = BoardState.fromGame(game)
print([str(player) for player in game.players])
print([str(domino) for domino in game.players[0].hand])
print(Train.startingSide)
plays = bs.availablePlays(game.players[0])
print(plays)


['id:0 train:0', 'id:1 train:1', 'id:2 train:2', 'id:3 train:3', 'id:4 train:4', 'id:5 train:5']
['(1, 3)', '(3, 10)', '(2, 5)', '(0, 1)', '(0, 6)', '(7, 10)', '(4, 4)', '(9, 11)', '(4, 11)', '(8, 8)', '(10, 12)', '(6, 8)']
11
[((9, 11), (0, 11)), ((4, 11), (0, 11))]


<h3>Testing random play</h3>

In [43]:
plays = bs.availablePlays(game.players[0])
if len(plays)> 0:
    play = plays[random.randint(0, len(plays) - 1)] # play = (dominoSideTuple,placementTuple)
    player = game.getPlayer(0)
    domino = player.getDominoFromSides(*play[0])
    train = bs.getTrain(play[1][0]) # play[1] is placement, = (train.id, side)
    print(train)
    player.play(domino,play[1][1],train)
    print(train)
else:
    print("no possible plays")

id: 0 trainUp?:False openSides: [12]
id: 0 trainUp?:False openSides: [4]


<h1>Start Making Env</h1>

In [2]:
# %load DominoEnv.py
from classes import Game, BoardState, Train, Player, Domino
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Dict, Discrete , MultiDiscrete, Box, Sequence, utils, Tuple
import numpy as np
import random
class DominoTrainEnv(Env):
    def __init__(self,numPlayers:int, maxFails:int):
        # Actions we can take, 13,13 for possible domino sides, [9,13] for possible domino placements
        #action space needed to be changed as it would cause an unknown issue with 
        #self.action_space = MultiDiscrete(np.array([[13, 13], [9, 13]])) 
        self.action_space = Box(low=np.array([[0, 0], [0, 0]]), high=np.array([[12, 12], [8, 12]]), dtype=np.int8 )
        # observation space
        obsv =  {
        "hand": Box(high=np.array([[13, 13]*79]), dtype=np.int8,low=np.array([[-1, -1]*79])),
        "placements": Box(high=np.array([[9, 13]*208]), dtype=np.int8,low=np.array([[-1,-1]*208])),
        "trains": Box(high=np.array([[9, 2]*8]), dtype=np.int8,low=np.array([[-1,-1]*8]))
        }
        self.observation_space = Dict(obsv)
        #setup game
        self.game = Game(numPlayers)
        self.logs = ""
        self.player = self.game.getPlayer(0)
        state = self.getState()
        self.state = state
        self.fails = 0
        self.maxFails = maxFails
        if self.game.startingPlayer != 0:
            bs = BoardState.fromGame(self.game)
            self.playOthers(bs,self.game.startingPlayer)
    def playOthers(self, bs:BoardState, start_index):
        stateChanged = False
        players = self.game.players
        for i in range(start_index, len(players)):
            player = players[i]
            posActions = bs.availablePlays(player)
            self.log(f"current turn: {player}, plays: {posActions}")
            if len(posActions)<=0:
                if not player.pickup(self.game.boneyard): self.game.end()
                posActions = bs.availablePlays(player)
            if len(posActions)>0:
                ranIndex = 0
                if len(posActions)>1: ranIndex = random.randint(0, len(posActions)-1)
                ranAction = posActions[ranIndex]
                ranDomino = player.getDominoFromSides(*ranAction[0])
                self._play(ranDomino,ranAction[1],player,bs)
                stateChanged = True
        return stateChanged
    @staticmethod
    def _padArray(array,len:int):
        return np.pad(array,((0,len),(0,0)),mode='constant',constant_values=-1)
    def getState(self):
        bs = BoardState.fromGame(self.game)
        handarray = np.array([domino.sides for domino in self.player.hand], dtype=np.int8)
        placements = np.array(bs.getPlacements(), dtype=np.int8)
        train_info = np.array([[train.id, int(train.trainUp)] for train in [*bs.trains,bs.mexican]], dtype=np.int8)
        self.log(handarray.shape)
        hand_padding = DominoTrainEnv._padArray(handarray, 79-len(handarray))
        placement_padding = DominoTrainEnv._padArray(placements, 208-len(placements))
        train_padding = DominoTrainEnv._padArray(train_info, 8-len(train_info))
        state =  {
            "hand": hand_padding.ravel().reshape((1,158)),
            "placements": placement_padding.ravel().reshape((1,416)),
            "trains": train_padding.ravel().reshape((1,16))
        }
        self.log(state)
        return state
    def _play(self,domino:Domino, placement,player:Player, bs:BoardState):
        self.log(f"attempting to play {domino} on {placement} from {player}")
        game = self.game
        played = False
        if domino is not None:
            train = game.getTrain(placement[0])
            if player.play(domino,placement[1],train) is not False:
                played = True   
                if domino.isDouble:
                    self.log("double played")
                    players = game.players
                    start_index = players.index(player)
                    #loop through all players at "table" starting with person who played double
                    for i in range(len(players)):
                        index = (start_index + i) % len(players)
                        loop_player = players[index] 
                        newPlacement = (placement[0],domino.sides[0])
                        plays = bs.availablePlays(loop_player, placements=[placement])
                        self.log(f"checking if {loop_player} can play on {newPlacement}\n, plays {plays}")
                        #if player can't play on double
                        if len(plays)<=0:
                            pickupDomino = loop_player.pickup(self.game.boneyard)
                            #if a domino was actually picked up
                            if pickupDomino:
                                self.log(f"{loop_player} pickedup {pickupDomino}")
                                plays = bs.availablePlays(loop_player, placements=[newPlacement])
                                if len(plays)<=0: 
                                    self.log("player can't play pickup")
                                    loop_player.train.trainUp = True
                                    continue
                                #if possible to play pickup
                                else:
                                    self.log(f"attepmting to play pickup on {newPlacement}, train {train}")
                                    if loop_player.play(pickupDomino,newPlacement[1],train) is None:
                                        game.done = True
                                    break
                                    
                            # no domino was pickedup, meaning boneyard is empty and end of game
                            else:
                                game.done = True
                                break
                        #if player only has one choice to play
                        elif len(plays) == 1:
                            play = plays[0]
                            ranDomino = loop_player.getDominoFromSides(*play[0])
                            self.log(f"attempting to play {ranDomino} on {newPlacement}, player.train: {player.train}")
                            if loop_player.play(ranDomino,newPlacement[1],train) is None:
                                game.done = True
                            break
                        #if player has many choices to play
                        else:
                            #if ai player
                            if loop_player.id == 0:
                                self.log(f"letting ai make choice for double play")
                                self.game.unsastifiedDouble = (train.id,domino.sides[0])
                                self.game.prevPlayer = player.id
                                break
                            #if other players, random choice
                            else:
                                self.log(player.train)
                                play = plays[random.randint(0, len(plays) - 1)]
                                ranDomino = loop_player.getDominoFromSides(*play[0])
                                self.log(f"attempting to play {ranDomino} on {newPlacement}, player.train: {player.train}\n available plays: {plays}")
                                if loop_player.play(ranDomino,newPlacement[1],train) is None:
                                    game.done = True
                            break
            else:
                self.log("Invalid Placement")
        else:
            self.log("Invlaid Domino")
        if played and len(player.hand) == 0:
            self.game.end(player.id)
        return played

    def convertAction(self, action):
        return action
    def step(self, action):
        action = self.convertAction(action)
        stateChanged = False
        reward = 0
        bs= BoardState.fromGame(self.game)
        if bs.isValidPlay(self.player,action):
            if bs.unsastifiedDouble is not None:
                domino = action[0]
                domino = self.player.getDominoFromSides(*domino)
                self._play(domino,action[1],self.player,bs)
                reward += domino.calc_points()
                stateChanged = True
                bs.unsastifiedDouble = None #probably a better way of doing this
                self.game.unsastifiedDouble = None
            else:    
                player = self.player
                posActions = bs.availablePlays(player)
                #if no action available
                if len(posActions)<=0:
                    if not self.player.pickup(self.game.boneyard): self.game.end()
                    posActions = bs.availablePlays(self.player)
                    if len(posActions)>0:
                        ranAction = posActions[0]
                        ranDomino = self.player.getDominoFromSides(*ranAction[0])
                        self._play(ranDomino,ranAction[1],self.player,bs)
                        stateChanged = True
                else:
                    # Apply action
                    domino = action[0]
                    domino = self.player.getDominoFromSides(*domino)
                    self._play(domino,action[1],self.player,bs)
                    reward += domino.calc_points()
                    stateChanged = True
            if stateChanged: bs = BoardState.fromGame(self.game)
            self.playOthers(bs=bs,start_index=self.game.nextPlayer(self.player.id))
            reward += 10 
        #invalid action
        else: 
            self.log("Invalid Action from step call")
            reward += -50
            self.fails +=1
        done = self.game.done
        if stateChanged and not done:    
            self.state = self.getState()
        truncated = False
        #hard limit on game length since, the ai can make invalid plays which could loop forever
        if self.fails >=self.maxFails:
            done = True
            truncated = True
        elif done:
            # add negative reward for points remaining in hand at game end
            reward += -1*self.player.pointsInHand() 
            if self.game.winner ==0:
                reward += 100
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done,truncated, info
    def render(self):
        # Implement viz
        print(self.logs)
        self.logs = ""
    def log(self, log):
        self.logs += str(log)+ "\n"
    def log_to_file(self, filename='log.txt'):
        try:
            with open(filename, 'a') as file:
                file.write(self.logs + '\n\n\n')
        except Exception as e:
            print(f"Error: {e}")
    def reset(self, seed=None, options=None):
        self.game = Game(self.game.numPlayers)
        self.player = self.game.getPlayer(0)
        self.state = self.getState()
        self.fails = 0
        self.log("------RESET------")
        self.log_to_file()
        self.logs = ""
        random.seed(seed)
        np.random.seed(seed)
        return self.state, {}
class DominoTrainEnvMaskable(DominoTrainEnv):
    def __init__(self,numPlayers:int,maxFails:int):
        # Actions we can take, 13,13 for possible domino sides, [9,13] for possible domino placements
        super().__init__(numPlayers,maxFails)
        self.actions = []
        allDominos = [(side1,side2) for side1 in range(0,13) for side2 in range(0,13)]
        placements = [(train,side) for side in range(0,13) for train in range(0,9)]
        for domino in allDominos:
           for placement in placements:
               self.actions.append((domino,placement))
        self.actions = tuple(self.actions)
        self.action_space = Discrete(19773)
        #self.action_space = MultiDiscrete([13, 13, 9, 13], dtype=np.int8)
    def getMaskMultiDiscrete(self):
        #[[[0-1x13],[0-1x13]],[[0-1x8],[0-1x13]]]
        #[[13],[13],[9],[13]]
        bs = BoardState.fromGame(self.game)
        mask = []
        plays = bs.availablePlays(self.player)
        if len(plays)==0:
            ones13 =  np.ones(13,dtype=np.bool)
            ones9 = np.ones(9,dtype=np.bool)
            mask = [ones13,ones13,ones9,ones13]
        else:
            zero13 =  np.zeros(13,dtype=np.bool)
            zero9 = np.zeros(9,dtype=np.bool)
            mask = [zero13,zero13,zero9,zero13]
            for action in bs.availablePlays(self.player):
                domino = action[0]
                placement = action[1]
                mask[0][domino[0]-1] = True
                mask[1][domino[1]-1] = True
                mask[2][placement[0]-1] = True
                mask[3][placement[1]-1] = True
        return mask
    def getMaskDiscrete(self):
        bs = BoardState.fromGame(self.game)
        mask = np.zeros(19773)
        for action in bs.availablePlays(self.player):
            index = self.actions.index(action)
            mask[index] = 1
        return mask
    def convertAction(self, action):
        return self.actions[action] # converts index into action
        #return [[action[0],action[1]],[action[2],action[3]]]    
    
    #def getState(self):
    #    bs = BoardState.fromGame(self.game)
    #    handarray = np.array([domino.sides for domino in self.player.hand], dtype=np.int8)
    #    placements = np.array(bs.getPlacements(), dtype=np.int8)
    #   train_info = np.array([[train.id, int(train.trainUp)] for train in bs.trains], dtype=np.int8)
    #    
    #   hand_padding = DominoTrainEnv._padArray(handarray, 79-len(handarray))
    #    placement_padding = DominoTrainEnv._padArray(placements, 208-len(placements))
    #    train_padding = DominoTrainEnv._padArray(train_info, 8-len(train_info))
    #    state =  (
    #        hand_padding.ravel().reshape((1,158)),
    #        placement_padding.ravel().reshape((1,416)),
    #        train_padding.ravel().reshape((1,16))
    #    )
     #   return state

<h2>Test ENV</h2>

<h3>Random Sample Test</h3>

In [49]:
env = DominoTrainEnvMaskable(6,100)

actions length:  19773


In [50]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    step = 0
    while not done:
        #env.render()
        #print(step)
        step+=1
        action = env.action_space.sample()
        n_state, reward, done, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    print(step)

8357
9240
2625
9801
15533
11339
2228
10438
8595
14611
1535
2166
8662
4591
8084
7908
15839
15118
1716
15103
8601
2088
5571
3225
9177
5884
1005
11619
7141
8864
691
17980
11043
15039
5623
175
14489
48
9133
7424
457
8768
18074
16088
17818
17474
3830
4961
13229
6141
14090
4816
9749
11168
16426
1592
1103
245
7216
2142
15613
5275
9850
4948
18020
5413
6687
8541
13820
16626
16297
685
17044
15154
18192
17633
19273
17421
3184
624
19692
11738
1812
792
5727
8414
1298
13752
6685
16827
3521
7287
522
2990
3299
10107
9328
14271
12879
11412
Episode:1 Score:-5000
100
18015
12343
1620
11389
148
12994
492
7374
12558
16765
11004
5619
2031
16837
13030
115
3125
18270
16382
12430
13423
18427
6707
19291
3299
11388
16846
2774
3535
12761
17045
14425
5432
15645
14438
13635
2976
10767
19550
13395
10827
19450
7641
7632
19063
4932
14357
10246
12802
5745
15622
12837
19299
215
14732
2224
11487
16288
13333
12065
14257
11607
10126
7850
201
11640
16291
14141
5997
5340
14742
17924
302
9405
18820
3460
7662
13084
98
14379
13

<h3>Env checker</h3>

In [14]:
import gymnasium as gym
from stable_baselines3.common.env_checker import check_env
env = DominoTrainEnv(6)
#print(gym.spaces.flatdim(env.observation_space))
print(env.observation_space.shape)
print(env.observation_space)
print(type(env.observation_space).__base__)
print(isinstance(env.observation_space, gym.spaces.Space))
check_env(env,skip_render_check=True)

None
Dict('hand': Box(-1, 13, (1, 158), int8), 'placements': Box(-1, [[ 9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13  9 13
   9 13  9 13  9 13  9 13  9 13  9 13  9 13

<h1>Make RL Model</h1>

<h2>Intial Attempt</h2>

In [3]:
import numpy as np
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
env = DominoTrainEnv(6)

# Initialize the PPO agent
#model = PPO("MultiInputPolicy", env,verbose=1)

# Train the agent

# Save the trained model

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = PPO.load("PPO_DominoTrain", env=env,verbose=1)
model.learn(total_timesteps=int(5e5), progress_bar=True)
model.save("PPO_DominoTrain")
model = PPO.load("PPO_DominoTrain", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print(mean_reward, std_reward)
# Load the saved model
#model = PPO.load("PPO_DominoTrain")

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501,698/500,000  [ 0:19:57 < 0:00:00 , 433 it/s ]

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-500.0 0.0


<h2>debugging minimal reproducable</h2>

In [ ]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

class TestEnv(Env):
    def __init__(self,numPlayers:int):
        # Actions we can take, 13,13 for possible domino sides, [9,13] for possible domino placements
        #self.action_space = MultiDiscrete(np.array([[13, 13], [9, 13]]))
        self.action_space = Box(low=np.array([[0, 0], [0, 0]]), high=np.array([[12, 12], [8, 12]]), dtype=np.int8 )
        # observation space
        obsv =  {
        "hand": Box(high=np.array([[13, 13]*79]), dtype=np.int8,low=np.array([[-1, -1]*79]))
        }
        self.observation_space = Dict(obsv)
        #self.observation_space = Box(high=np.array([[13, 13]*79]).ravel().reshape((1,158)), dtype=np.int8,low=np.array([[-1, -1]*79]).ravel().reshape((1,158)))
        self.state = self.getState()
    @staticmethod
    def __padArray(array,len:int):
        return np.pad(array,((0,len),(0,0)),mode='constant',constant_values=-1)
    def getState(self):
        #array values are placeholders
        handarray = np.array([(11,11),(12,11)], dtype=np.int8)
        
        hand_padding = TestEnv.__padArray(handarray, 79-len(handarray))
        state =  {
            "hand": hand_padding.ravel().reshape((1,158)),
        }
        #state = state["hand"]
        return state
    def step(self, action):
        self.state = self.getState()
        # Return step information
        reward = 1
        done = False
        info = {}
        return self.state, reward, done,False, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self, seed=None):
        self.state = self.getState()
        return self.state, {}

env = TestEnv(6)
print(check_env(env,skip_render_check=True))
model = PPO("MultiInputPolicy", env)

<h2>Action Masking</h2>

In [39]:
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.evaluation import evaluate_policy
from sb3_contrib.common.maskable.utils import get_action_masks
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

def mask_fn(env: gym.Env) -> np.ndarray:
    return env.getMaskDiscrete()


env = DominoTrainEnvMaskable(6, 100)  # Initialize env
env = ActionMasker(env, mask_fn)  # Wrap to enable masking

# MaskablePPO behaves the same as SB3's PPO unless the env is wrapped
# with ActionMasker. If the wrapper is detected, the masks are automatically
# retrieved and used when learning. Note that MaskablePPO does not accept
# a new action_mask_fn kwarg, as it did in an earlier draft.
model = MaskablePPO('MultiInputPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [33]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env


class CustomEnv(gym.Env):

    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Dict({"hand":spaces.Box(low=-1, high=1, shape=(6,))})
        self.action_space = spaces.MultiDiscrete(np.array([[1, 2], [3, 4]]), dtype=np.int8)

    def reset(self, seed=None, options=None):
        return self.observation_space.sample(), {}

    def step(self, action):
        obs = self.observation_space.sample()
        reward = 1.0
        terminated = False
        truncated = False
        info = {}
        return obs, reward, terminated, truncated, info

env = CustomEnv()
check_env(env)

model = PPO("MultiInputPolicy", env, verbose=1, device='cuda')

TypeError: Field elements must be 2- or 3-tuples, got '3'

In [52]:
MultiDiscrete([13, 13, 9, 8]).sample()

array([11,  5,  6,  5], dtype=int64)